`
SELECT 
bugId,
b.testerId,
fwl.description AS fixWorthinessLevel,
b.createDate,
b.subject,
REPLACE(actionPerformed,"\n"," ") as actionPerformed,
REPLACE(expectedResult,"\n"," ") as expectedResult,
REPLACE(result,"\n"," ") as result,
REPLACE(errorMessage,"\n"," ") as errorMessage
FROM utest30.Bug b
JOIN TestCycle tc ON b.testCycleId = tc.testCycleId
JOIN FixWorthinessLevelLocale fwl ON fwl.fixWorthinessLevelId = b.fixWorthinessLevelId
WHERE tc.productId = 8311
AND b.bugStatusId = 2
AND b.createDate > '2017-01-01 00:00:00'
`

In [128]:
import re, graphlab
graphlab.canvas.set_target('ipynb')

In [129]:
reviews = graphlab.SFrame('gs_texts.csv')
reviews['createDate'] = reviews['createDate'].str_to_datetime()
reviews.head(n=1)

Finished parsing file /home/bo/pyml/projects/thom pocs/bug clustering/gs_texts.csv

Parsing completed. Parsed 100 lines in 0.025451 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,int,str,str,str,str,str,str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Read 842 lines. Lines per second: 24305.1

Finished parsing file /home/bo/pyml/projects/thom pocs/bug clustering/gs_texts.csv

Parsing completed. Parsed 842 lines in 0.03479 secs.

bugId 
 testerId 
 fixWorthinessLevel 
 createDate 
 subject 
 actionPerformed 
 
 
 2409447 
 144476 
 somewhat valuable 
 2017-01-04 01:56:32 
 [Fn]Zone picker is not fu nctional[Zonepicker][ ... 
 1. Launch browosony url. 2. Enter zip code & ... 
 
 
 
 
 expectedResult 
 result 
 errorMessage 
 
 
 It should redirect to home page. ... 
 zone picker is not functional ... 
 None 
 
 
[1 rows x 9 columns]

In [130]:
reviews['text'] = reviews['subject'] + ' ' + reviews['result']

In [131]:
def rm_char(text):
    if text:
        return re.sub(r'\W+', ' ', text)
    else:
        return ""

In [132]:
reviews['cleaned'] = reviews.apply(
    lambda x: rm_char(x['text'])
)

In [133]:
reviews.head(n=1)

bugId 
 testerId 
 fixWorthinessLevel 
 createDate 
 subject 
 actionPerformed 
 
 
 2409447 
 144476 
 somewhat valuable 
 2017-01-04 01:56:32 
 [Fn]Zone picker is not fu nctional[Zonepicker][ ... 
 1. Launch browosony url. 2. Enter zip code & ... 
 
 
 
 
 expectedResult 
 result 
 errorMessage 
 text 
 cleaned 
 
 
 It should redirect to home page. ... 
 zone picker is not functional ... 
 None 
 [Fn]Zone picker is not fu nctional[Zonepicker][ ... 
 Fn Zone picker is not functional Zonepicker ... 
 
 
[1 rows x 11 columns]

# Explore topics

In [134]:
reviews['bow'] = graphlab.text_analytics.count_words(reviews['cleaned'])

In [135]:
reviews['bow'] = reviews['bow'].dict_trim_by_keys(graphlab.text_analytics.stopwords() | set([]), exclude=True)

In [136]:
topic_model = graphlab.topic_model.create(reviews['bow'], num_topics=10, num_iterations=300, verbose=False, print_interval=100)

Learning a topic model

Number of documents 842

Vocabulary size 1597

Running collapsed Gibbs sampling

+-----------+---------------+----------------+-----------------+

...

+-----------+---------------+----------------+-----------------+

| 100 | 415.143ms | 2.40528e+06 | 0 |

| 200 | 857.187ms | 1.12947e+06 | 0 |

| 300 | 1.30s | 2.64262e+06 | 0 |

+-----------+---------------+----------------+-----------------+

In [137]:
i = 0
topics = topic_model.get_topics(output_type='topic_words')
for row in topics:
    print i, row['words']
    i += 1

0 ['list', 'shopping', 'item', 'cart', 'added']
1 ['store', 'showing', 'products', 'category', '1']
2 ['displayed', 'error', 'message', 'android', 'items']
3 ['app', 'address', 'android', 'crashes', 'mode']
4 ['fn', 'checkout', 'unable', 'items', 'account']
5 ['page', 'fn', 'button', 'web', 'home']
6 ['page', 'fn', 'order', 'orders', 'button']
7 ['fn', 'membership', 'web', 'user', 'card']
8 ['mac', 'promo', 'option', 'clicking', 'displaying']
9 ['product', 'image', 'missing', 'pdp', 'web']


In [138]:
reviews['topic'] = topic_model.predict(reviews['bow'])

# Top bugs of Google shopping express

In [139]:
reviews['topic'].show(view='Categorical')

# Drill down to the issue

In [140]:
reviews['topic_probabilities'] = topic_model.predict(reviews['bow'], output_type='probability')

In [141]:
reviews['topic_probability'] = reviews['topic_probabilities'].apply(lambda x: max(x))

In [142]:
# for each topic, print 3 top reviews
topic_count = {i: 0 for i in range(len(topics))}
top_reviews = []
for r in reviews.sort('topic_probability', ascending=False)[:1000]:
    tp = r['topic']
    if topic_count[tp] < 5:
        top_reviews.append((r['topic'], r))
        topic_count[tp] += 1
        
top_reviews = sorted(top_reviews, key=lambda x:int(x[0]))
for tp, r in top_reviews:
    print 'Topic(%s | %s | %s |%s |%s): %s' % (tp, r['topic_probability'], r['fixWorthinessLevel'], r['testerId'], r['createDate'], topics[tp]['words'])
    print r['subject']
    print r['result']
    print

Topic(0 | 0.323943661972 | very valuable |142608 |2017-04-12 03:20:45): ['list', 'shopping', 'item', 'cart', 'added']
[Fn] Unable to edit item in shopping list. Edit button does not work [Shopping list][iPad/Safari]
Unable to edit item in shopping list. Edit button does not work

Topic(0 | 0.319444444444 | somewhat valuable |144476 |2017-04-19 02:04:54): ['list', 'shopping', 'item', 'cart', 'added']
[Fn]Item added to shopping list pop up is not displayed when item is added[Shopping list][Web]
Item added to shopping list pop up is not displayed when item is added to shopping list

Topic(0 | 0.279411764706 | somewhat valuable |142608 |2017-02-17 02:51:49): ['list', 'shopping', 'item', 'cart', 'added']
[Fn] "Share list" appears and disappears in Shopping list menu [Shopping list][Android]
"Share list" appears and disappears in Shopping list menu

Topic(0 | 0.257575757576 | somewhat valuable |154880 |2017-02-24 05:40:13): ['list', 'shopping', 'item', 'cart', 'added']
[Fn] The same Item can

# What is the current top issues? (2017.6)

In [143]:
this_month_reviews = reviews[reviews['createDate'] > 1496275200] # 2017.06.01
this_month_reviews.head()

bugId,testerId,fixWorthinessLevel,createDate,subject
2665572,297110,somewhat valuable,2017-06-01 01:04:54,[fn][Complete shoppinglist name is not ...
2665618,241963,somewhat valuable,2017-06-01 02:00:39,[Fn] Auto suggestions arenot Showing While typing ...
2665619,241956,somewhat valuable,2017-06-01 02:02:32,[Fn]Unable to displayaddress added for the ...
2667338,241956,very valuable,2017-06-02 01:34:32,[Fn]App is crashed afterselecting undo for the ...
2667343,192264,somewhat valuable,2017-06-02 01:37:29,[Fn][Error at Shopping inzip-code and not ...
2667381,192264,somewhat valuable,2017-06-02 02:40:11,[Fn] App able to takeinvalid zipcode [Zone ...
2673452,242313,somewhat valuable,2017-06-06 01:02:55,FUN]Debug warning messageis diplayed while adding ...
2673460,156925,somewhat valuable,2017-06-06 01:18:03,[Tn][AssertnotNull errorwhen tapped on Join Now ...
2673496,173248,very valuable,2017-06-06 01:44:08,"[Fn] Cart & Checkout popup is not seen, unless ..."
2673511,156925,somewhat valuable,2017-06-06 01:56:29,[Fn][3dot menu missingfor a list that is ...


In [144]:
this_month_reviews['topic'].show(view='Categorical')

In [145]:
# for each topic, print 3 top reviews
topic_count = {i: 0 for i in range(len(topics))}
top_reviews = []
for r in reviews[reviews['createDate'] > 1496275200].sort('topic_probability', ascending=False):
    tp = r['topic']
    if topic_count[tp] < 3:
        top_reviews.append((r['topic'], r))
        topic_count[tp] += 1
        
top_reviews = sorted(top_reviews, key=lambda x:int(x[0]))
for tp, r in top_reviews:
    print 'Topic(%s | %s | %s |%s |%s): %s' % (tp, r['topic_probability'], r['fixWorthinessLevel'], r['testerId'], r['createDate'], topics[tp]['words'])
    print r['text']
    print

Topic(0 | 0.208333333333 | somewhat valuable |169077 |2017-06-09 05:11:49): ['list', 'shopping', 'item', 'cart', 'added']
[Fn]Added items to shopping list are not showing for guest user account[shopping list][iOS] 1.Not able to see added items to shopping list after adding items (or) products to shopping list.

Topic(0 | 0.203125 | somewhat valuable |241956 |2017-06-09 02:00:17): ['list', 'shopping', 'item', 'cart', 'added']
[Fn]Eventhough item is removed from list it is available in shop your list page[android] even though item is deleted it is available in shop your list pge

Topic(0 | 0.16393442623 | somewhat valuable |297110 |2017-06-01 01:04:54): ['list', 'shopping', 'item', 'cart', 'added']
[fn][Complete shopping list name is not displaying in SL header][Shopping List][Android] "S..." is displaying

Topic(1 | 0.243902439024 | somewhat valuable |297150 |2017-06-08 03:51:46): ['store', 'showing', 'products', 'category', '1']
[GUI][Most of the product images are missing in the categ

# Applications
1. DX feedback ALX => to customer
2. DX journey with focus
3. Sales tool